# Bread Board Constrcut record.py

In [ ]:
import os

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
    
from scipy import signal

from pyaspect.moment_tensor import MomentTensor
from pyaspect.specfemio.record import Record
import pyaspect.specfemio.record as record
from pyaspect.specfemio.read import _read_headers

## paths

In [ ]:
data_in_dir  = 'data/output/'
data_out_dir = data_in_dir
!ls {data_out_dir}/tmp/TestProjects/CGFR_Test

In [ ]:
projects_fqp = os.path.join(data_out_dir,'tmp','TestProjects','CGFR_Test')
#recip_project_fqp = os.path.join(projects_fqp,'ReciprocalGeometricTestProject') #geometric
#fwd_project_fqp = os.path.join(projects_fqp,'ForwardGeometricTestProject')      #geometric
recip_project_fqp = os.path.join(projects_fqp,'ReciprocalTestProject') #Groningen
fwd_project_fqp = os.path.join(projects_fqp,'ForwardTestProject')      #Groningen
!ls {recip_project_fqp}
print()
!ls {fwd_project_fqp}/run0001/OUTPUT_FILES/plot_source_time_function.txt

## Read Source-Time function

In [ ]:
of_fqp = os.path.join(fwd_project_fqp,'run0001','OUTPUT_FILES')
stf_fqp = os.path.join(of_fqp,'plot_source_time_function.txt')
!ls {stf_fqp}

stf_pair = np.genfromtxt(stf_fqp,dtype=np.float64).transpose()
stf_amp = stf_pair[1,:].copy()
stf_time = stf_pair[0,:].copy()

print(stf_amp)
print(stf_time)

## Read Source-Time function from Reciprocal Force Simulations

In [ ]:
recip_of_fqp = os.path.join(recip_project_fqp,'run0001','OUTPUT_FILES')
recip_stf_fqp = os.path.join(recip_of_fqp,'plot_source_time_function.txt')
!ls {recip_stf_fqp}

recip_stf_pair = np.genfromtxt(recip_stf_fqp,dtype=np.float64).transpose()
recip_stf_amp = recip_stf_pair[1,:].copy()
recip_stf_time = recip_stf_pair[0,:].copy()

print(recip_stf_amp)
print(recip_stf_time)

## Instantiate Record, and test slicing and pandas operations with DataFrames

In [ ]:
recip_record_fqp = os.path.join(recip_project_fqp,'pyheader.project_record')
recip_record = Record(recip_record_fqp)

ne = recip_record.nevents
ns = recip_record.nsrc


print(f'ne:{ne}, ns:{ns}')
print(f'recip: {recip_record.is_reciprocal}')
print(f'Recip Header:\n{recip_record.solutions_df.loc[pd.IndexSlice[:,1],:]}')
print(f'Recip Header:\n{recip_record}')

## create Reciprocal Green's Table (as DataFrame)

In [ ]:
def calulate_spacial_derivative(tdf,eidx,sidx,tidx,g_p1,g_m1,sos,comp_key,coord_key):
    gidx_0  = pd.IndexSlice[eidx,sidx,tidx,0]
    gidx_p1 = pd.IndexSlice[eidx,sidx,tidx,g_p1]
    gidx_m1 = pd.IndexSlice[eidx,sidx,tidx,g_m1]
    df_0    = tdf.loc[gidx_0]
    df_p1   = tdf.loc[gidx_p1]
    df_m1   = tdf.loc[gidx_m1]
    data_p1 = signal.sosfilt(sos, df_p1[comp_key].astype(np.float64))
    data_m1 = signal.sosfilt(sos, df_m1[comp_key].astype(np.float64))
    c_p1    = df_p1[coord_key]
    c_m1    = df_m1[coord_key]
    c_0     = df_0[coord_key]
    delta   = 0.5*(c_p1 - c_m1)
    h       = 2.0*np.abs(delta)
    c       = c_m1 + delta

    assert h != 0
    assert c_0-c == 0

    h_scale  = 1/h
    mt_trace = h_scale*(data_p1 - data_m1)

    return mt_trace

In [ ]:
def make_rgf_data(record,fl,fh,fs):

    comp_dict = {'comp_EX':0,'comp_NY':1,'comp_Z':2}
    coord_dict = {0:'lon_xc',1:'lat_yc',2:'depth'}
    sos = signal.butter(3, [fl,fh], 'bp', fs=fs, output='sos')

    l_rgf_traces = []
    m_df = pd.merge(record.stations_df,record.data_df,on=['eid','sid','trid','gid'])
    for eidx, edf in m_df.groupby(level='eid'):
        for sidx, sdf in edf.groupby(level='sid'):
            for tidx, tdf in sdf.groupby(level='trid'):
                for comp_key in comp_dict.keys():
                    ie = tidx
                    ig = eidx
                    fi = sidx
                    for di in range(3):
                        rgf_dict = {'eid':tidx,'trid':eidx,'fid':sidx}
                        rgf_dict['cid'] = comp_dict[comp_key]
                        coord_key = coord_dict[di]
                        rgf_dict['did'] = di
                        ip1 = di+1     #coord + h
                        im1 = ip1 + 3  #coord - h
                        if di == 2:
                            tm1 = ip1
                            ip1 = im1
                            im1 = tm1
                        rgf_dict['data'] = calulate_spacial_derivative(m_df,
                                                                       eidx,
                                                                       sidx,
                                                                       tidx,
                                                                       ip1,
                                                                       im1,
                                                                       sos,
                                                                       comp_key,
                                                                       coord_key)

                        rgf_dict['component'] = comp_key
                        l_rgf_traces.append(rgf_dict)

    return pd.DataFrame.from_records(l_rgf_traces, index=('eid','trid','cid','fid','did'))

In [ ]:
f_low = 1.0
f_high = 100.0
nsamp = 1000
rgf_data_df = make_rgf_data(recip_record,f_low,f_high,nsamp)
rgf_data_df

In [ ]:
#print(rgf_data_df)
#print(rgf_data_df.loc[0,0,0,:,:])
rgf_data_df.loc[(0),:]

## Instantiate Forward Record

In [ ]:
fwd_record_fqp = os.path.join(fwd_project_fqp,'pyheader.project_record')
fwd_record = Record(fwd_record_fqp)
data = fwd_record.data_df
print(data.loc[((1, 0, 0, 0), 'comp_EX')])

## Get Moment tensors to compare with Foward data and also Construct Combinded Reciprocal CMTs. These functions will not be part of record.py module, but make_moment_tensor will be added to utils.py module

In [ ]:
def make_moment_tensor(src_h):
    
    mrr = src_h['mrr']
    mtt = src_h['mtt']
    mpp = src_h['mpp']
    mrt = src_h['mrt']
    mrp = src_h['mrp']
    mtp = src_h['mtp']
    
    h_matrix = np.array([[mrr,mrt,mrp],[mrt,mtt,mtp],[mrp,mtp,mpp]])
    
    return MomentTensor(m_up_south_east=h_matrix)


#print(f'Forward Record Sources:\n{fwd_record_h.solutions_df}')
SrcHeader = fwd_record.solution_cls

d_fwd_src = {}
for eidx, edf in fwd_record.solutions_df.groupby(level='eid'):
    for sidx, sdf in edf.groupby(level='sid'):
        idx = pd.IndexSlice[eidx,sidx]
        src = SrcHeader.from_series(fwd_record.solutions_df.loc[idx])
        #print(src)
        #mag    = src.mw
        #strike = src.strike
        #dip    = src.dip
        #rake   = src.rake
        #mt = MomentTensor(mw=mag,strike=strike,dip=dip,rake=rake)
        mt = make_moment_tensor(src)
        print(mt)
        d_fwd_src[eidx] = mt
        #print(f'mt.aki_m6:\n{mt.aki_richards_m6()}')
        #print(f'header.m6:\n{src.mt}\n')

for key in d_fwd_src:
    print(d_fwd_src[key].m6_up_south_east())

In [ ]:
def calc_composite_recip_cmt_trace(eid,trid,mt_arr,rgf_df,force_stf,cmt_stf):

    comp_dict = {'comp_EX':0,'comp_NY':1,'comp_Z':2}
    
    wzz =  mt_arr[0] #mrr
    wyy =  mt_arr[1] #mtt
    wxx =  mt_arr[2] #mpp
    wyz = -mt_arr[3] #mrt
    wxz =  mt_arr[4] #mrp
    wxy = -mt_arr[5] #mtp

    cmt_trace_dict = {'eid':eid, 'trid':trid}
    for comp_key in comp_dict.keys():
        ic = comp_dict[comp_key]

        composite_trace  = wxx*rgf_df.loc[(eid,trid,0,ic, 0),'data'].copy() #Matrix: Mee
        composite_trace += wyy*rgf_df.loc[(eid,trid,1,ic, 1),'data'] #Matrix: Mnn
        composite_trace += wzz*rgf_df.loc[(eid,trid,2,ic, 2),'data'] #Matrix: Mzz

        #Matrix: M1/Mxy
        composite_trace += wxy*rgf_df.loc[(eid,trid,1,ic, 0),'data']
        composite_trace += wxy*rgf_df.loc[(eid,trid,0,ic, 1),'data']

        #Matrix: M2/Mxz
        composite_trace += wxz*rgf_df.loc[(eid,trid,0,ic, 2),'data']
        composite_trace += wxz*rgf_df.loc[(eid,trid,2,ic, 0),'data']

        #Matrix: M3/Myz
        composite_trace += wyz*rgf_df.loc[(eid,trid,1,ic, 2),'data']
        composite_trace += wyz*rgf_df.loc[(eid,trid,2,ic, 1),'data']


        #deconvolve and then convolved
        deconv = 1.0/force_stf[0]
        scaled_trace = deconv*np.convolve(composite_trace.astype(np.float64),cmt_stf.astype(np.float64))[:len(cmt_stf)]

        # convert back to single precision
        cmt_trace_dict[comp_key] = scaled_trace.astype(np.float32)
        print(f'dict[{comp_key}]:\n{type(cmt_trace_dict[comp_key])}')
        
        
    #return cmt_trace_dict
    #return pd.DataFrame.from_records([cmt_trace_dict], index=("eid","trid"))
    #print(f'tdf:\n{pd.Series(cmt_trace_dict).to_frame().T}')
    #.T.set_index(["eid","trid"],inplace=True)}')
    #print(f'series["comp_EX"]: {type(pd.Series(cmt_trace_dict).to_frame().loc["comp_EX"].values)}')
    #print(f'series["comp_EX"]: {type(pd.Series(cmt_trace_dict)["comp_EX"])}')
    tdf = pd.Series(cmt_trace_dict).to_frame().T
    #print(f'type(tdf["comp_EX"]): {type(tdf["comp_EX"].values)}')
    #print(f'tdf["comp_EX"]: {tdf["comp_EX"].values[0]}')
    #return tdf
    return pd.Series(cmt_trace_dict)

In [ ]:
def calc_composite_recipt_cmt_traces_for_event(eid,mt,rgf_df,force_stf,cmt_stf):
    
    edf = rgf_df.loc[eid]
    ntr = edf.index.get_level_values('trid').nunique()
    
    mw = mt.magnitude
    m0 = mt.moment
    mt_arr = mt.m6_up_south_east()

    l_recip_cmt_traces = []
    #for tidx, tdf in rgf_df.groupby(level='trid'):
    imerge = 0
    edf = None
    for tidx in range(ntr):

        #slice_idx = pd.IndexSlice[eidx,tidx, 0,ic, 0]
        #slice_rgf_df = rgf_df.loc[slice_idx]
        #d_recip_cmt = calc_composite_recip_cmt_trace(eid,tidx,mt_arr,tdf,force_stf,cmt_stf)
        #d_recip_cmt = calc_composite_recip_cmt_trace(eid,tidx,mt_arr,edf.loc[tidx],force_stf,cmt_stf)
        #d_recip_cmt = calc_composite_recip_cmt_trace(eid,tidx,mt_arr,edf.loc[tidx],force_stf,cmt_stf)
        tseries = calc_composite_recip_cmt_trace(eidx,tidx,mt_arr,rgf_data_df,recip_stf_amp,stf_amp)
        edf = pd.concat([edf,tseries.to_frame().T])
        print(f'type(edf): {type(edf)}')

        # add the mapped dictionary to a list so that it can be converted to a pd.DataFrame
        #l_recip_cmt_traces.append(d_recip_cmt)
        '''
        if imerge == 0:
            tdf = d_recip_cmt.to_frame().T.set_index(['eid','trid'],inplace = True)
        else:
            tdf = pd.concat([tdf,d_recip_cmt])
        ''';
        
        #imerge += 1
        
    #return l_recip_cmt_traces
    #return pd.DataFrame.from_records(l_recip_cmt_traces, index=("eid","trid"))
    edf.set_index(['eid','trid'],inplace=True)
    #print(f'edf:\n{edf}')
    return edf

In [ ]:
def make_cmt_data_from_rgf(rgf_df,mt_dict,force_stf,cmt_stf):

    comp_dict = {'comp_EX':0,'comp_NY':1,'comp_Z':2}

    rgf_events = list(rgf_df.index.get_level_values('eid').unique())
    ne = rgf_df.index.get_level_values('eid').nunique()
    #print(f'rgf_events: {rgf_events}')
    mt_events  = list(mt_dict.keys())
    #print(f'mt_events: {mt_events}')
    #print(f'all: {rgf_events == mt_events}')

    if not rgf_events == mt_events:
        raise Exception('RGF-events do not match MomentTensors-events')

    l_recip_cmt_traces = []
    #for eidx, edf in rgf_df.groupby(level='eid'):
    imerge = 0
    edf = None
    for eidx in range(ne):

        #print(f'edf:\n{edf}')
        #print(f'edf.loc[pd.Islice[{eidx}]]:\n{rgf_df.loc[pd.IndexSlice[0]]}')
        #print(f'edf[{eidx}]:\n{rgf_df[eidx]}')
        #l_recip_cmt_traces += calc_composite_recipt_cmt_traces_for_event(eidx,mt_dict[eidx],edf,force_stf,cmt_stf)
        #l_recip_cmt_traces += calc_composite_recipt_cmt_traces_for_event(eidx,mt_dict[eidx],rgf_df,force_stf,cmt_stf)
        #l_tmp = calc_composite_recipt_cmt_traces_for_event(eidx,mt_dict[eidx],rgf_df,force_stf,cmt_stf)
        df = calc_composite_recipt_cmt_traces_for_event(eidx,mt_dict[eidx],rgf_df,force_stf,cmt_stf)
        if imerge == 0:
            #edf = pd.DataFrame.from_records(l_tmp, index=("eid","trid"))
            edf = df
        else:
            #df = pd.DataFrame.from_records(l_tmp, index=("eid","trid"))
            edf = pd.concat([edf,df])
        #l_recip_cmt_traces += l_tmp
        #print(f'tmp_df:\n{pd.DataFrame.from_records(l_tmp, index=("eid","trid"))}')
        
        imerge += 1

    return edf
    #return pd.DataFrame.from_records(l_recip_cmt_traces, index=('eid','trid'))

## Define function to make elementary moment-tensor traces

## Construct the Dataframe with the Reciprocal CMT Traces 

In [ ]:
#rgf_cmt_data_df = record.make_cmt_data_from_rgf(rgf_data_df,d_fwd_src,recip_stf_amp,stf_amp)
rgf_cmt_data_df = make_cmt_data_from_rgf(rgf_data_df,d_fwd_src,recip_stf_amp,stf_amp)

#rgf_cmt_data_df.loc[pd.IndexSlice[0,0,:,0],:]
#rgf_cmt_data_df.loc[pd.IndexSlice[0:1,:],'comp_EX']
#rgf_cmt_data_df.loc[(0,0),'comp_EX']
rgf_cmt_data_df
test_trace = calc_composite_recip_cmt_trace(0,0,np.array([1,0,0,0,0,0]),rgf_data_df,recip_stf_amp,stf_amp)
test_trace['comp_EX']

## Construct the Reciprocal CMT RecordHeader and then a Record

## Plot comparison between the Forward and Reciprocal CMT traces

In [ ]:
fig = plt.figure()
irow = 1
icol = 1
iplt = 1

# Filter Forward/CMT traces the same as the Recriprocal traces
sos = signal.butter(3, [f_low,f_high], 'bp', fs=nsamp, output='sos')

'''
for eidx, edf in composite_record.data_df.groupby(level='eid'):
    for sidx, sdf in edf.groupby(level='sid'):
        for tidx, tdf in sdf.groupby(level='trid'):
            for gidx, gdf in tdf.groupby(level='gid'):
''';
sidx = 0
gidx = 0
for eidx in rgf_data_df.index.get_level_values('eid').unique():
    mt_arr = d_fwd_src[eidx].m6_up_south_east()
    #r_trace = calc_elem_recip_cmt_traces_for_event(eidx,mt_arr,rgf_data_df,recip_stf_amp,stf_amp)
    #for tidx in range(len(r_trace)):
    for tidx in rgf_data_df.index.get_level_values('trid').unique():
        #r_trace = make_recip_cmt_from_elem_traces(mt_arr,tensor_traces[eidx,tidx],recip_stf_amp,stf_amp)
        #r_trace = make_recip_cmt_from_elem_traces(mt_arr,tensor_traces,eidx,tidx,recip_stf_amp,stf_amp)
        #print(f'edi: {eidx}')
        #print(f'c_trace.shape: {c_trace.shape}')
        ic = 0
        trace_df = calc_composite_recip_cmt_trace(eidx,tidx,mt_arr,rgf_data_df,recip_stf_amp,stf_amp)
        print(trace_df)
        for c in ['comp_EX','comp_NY','comp_Z']: #FIXME with a dictionary
            #print(type(composite_record.data_df.loc[(eidx,sidx,tidx,gidx),'comp_EX']))
            #c_trace = composite_record.data_df.loc[(eidx,sidx,tidx,gidx),c]
            #print(f'loc[{(eidx,sidx,tidx,gidx),c}]')
            print(f'rindex: {[(eidx,tidx),c]}')
            #r_trace = rgf_cmt_data_df.loc[(eidx,tidx),c]
            r_trace = trace_df[c]
            print(r_trace)
            f_trace = signal.sosfilt(sos,fwd_record.data_df.loc[(eidx,sidx,tidx,gidx),c])
            ax = fig.add_subplot(irow,icol,iplt)
            #ax.plot(c_trace,color='black',linestyle='-',linewidth=5,zorder=0,label='Recip',alpha=0.8)
            #ax.plot(c_trace,color='gold',linestyle='-',linewidth=2,zorder=1,label='Recip',alpha=0.8)
            ax.plot(r_trace,color='black',linestyle='-',linewidth=5,zorder=0,label='Recip',alpha=0.8)
            ax.plot(f_trace,color='gold',linestyle='--',linewidth=2,zorder=1,label='CMT')
            ax.set_title(f'Event:{eidx}, Trace:{tidx}, Comp:{c}')
            n = len(fig.axes)
            for i in range(n):
                fig.axes[i].change_geometry(n+1, 1, i+1)
            irow = n+1
            iplt = n+1
        ic += 1
                    
 
fig.set_figwidth(15)
fig.set_figheight(3*len(fig.axes))
fig.subplots_adjust(hspace=.35)
plt.show()